```{contents}
```
## **Agent Node in LangGraph**

An **Agent Node** in LangGraph is a specialized node designed to encapsulate **autonomous decision-making behavior**.
It integrates **LLM reasoning, tool usage, memory access, and control-flow decisions** into a single executable unit within a stateful graph.

---

### **1. Intuition**

Traditional nodes perform fixed transformations.
An **Agent Node behaves like a mini cognitive system**:

> **Perceive → Reason → Decide → Act → Update State**

This allows the graph to model **autonomous agents** that can plan, react, reflect, and collaborate.

---

### **2. Formal Definition**

An **Agent Node** is a node whose execution logic is:

* Driven by an LLM
* Conditioned on the current graph **state**
* Capable of invoking **tools**
* Capable of selecting the **next action or route**

---

### **3. Where Agent Nodes Are Used**

| Use Case               | Why Agent Node              |
| ---------------------- | --------------------------- |
| Task planning          | Decompose goals             |
| Research               | Search, analyze, synthesize |
| Operations             | Decide and execute actions  |
| Multi-agent systems    | Role-based reasoning        |
| Self-healing systems   | Diagnose and fix            |
| Interactive assistants | Adaptive responses          |

---

### **4. Internal Structure of an Agent Node**

```
State → Prompt Construction
      → LLM Reasoning
      → Tool Selection (optional)
      → Tool Execution (optional)
      → Decision Output
      → State Update
```

---

### **5. Basic Agent Node Example**

```python
from langgraph.graph import StateGraph, END
from langchain.chat_models import ChatOpenAI
from typing import TypedDict

class State(TypedDict):
    input: str
    decision: str

llm = ChatOpenAI()

def agent_node(state: State):
    prompt = f"Decide what to do with: {state['input']}"
    decision = llm.invoke(prompt).content
    return {"decision": decision}

builder = StateGraph(State)
builder.add_node("agent", agent_node)
builder.set_entry_point("agent")
builder.add_edge("agent", END)

graph = builder.compile()
graph.invoke({"input": "Find the weather and summarize it"})
```

---

### **6. Agent Node with Tool Use**

```python
from langchain.tools import tool

@tool
def search(query: str) -> str:
    return f"Result for {query}"

def agent(state):
    if "weather" in state["input"]:
        result = search(state["input"])
        return {"decision": result}
    return {"decision": "No tool used"}
```

---

### **7. Agent Node in a Multi-Agent System**

| Agent      | Responsibility        |
| ---------- | --------------------- |
| Planner    | Break task into steps |
| Researcher | Gather data           |
| Executor   | Perform actions       |
| Reviewer   | Verify correctness    |

Each is implemented as an **Agent Node**.

---

### **8. Variants of Agent Nodes**

| Variant          | Description               |
| ---------------- | ------------------------- |
| Reactive Agent   | Immediate response        |
| Planning Agent   | Multi-step planning       |
| Tool-Using Agent | Invokes external APIs     |
| Reflective Agent | Self-evaluates            |
| Supervisor Agent | Orchestrates other agents |
| Autonomous Agent | Self-directed execution   |

---

### **9. Agent Node + Control Flow**

Agent Nodes often control the graph using conditional edges:

```python
def route(state):
    if "done" in state["decision"]:
        return END
    return "agent"
```

This creates **agentic loops**.

---

### **10. Why Agent Nodes Matter**

| Without Agent Node | With Agent Node            |
| ------------------ | -------------------------- |
| Static workflows   | Autonomous systems         |
| Fixed behavior     | Adaptive behavior          |
| Manual pipelines   | Self-directing execution   |
| Brittle systems    | Robust intelligent systems |

---

### **11. Mental Model**

An **Agent Node** is:

> **An LLM-powered decision engine embedded inside a stateful execution graph**

It is the foundation for building **real AI systems** with planning, reasoning, and autonomy.


### Demonstration

In [3]:
from typing import TypedDict

class State(TypedDict):
    question: str
    answer: str
    satisfied: bool

from langchain.tools import tool

@tool
def improve(text: str) -> str:
    """Improves the given text."""
    return f"Improved version of: {text}"

from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

def agent_node(state: State):
    if not state["answer"]:
        answer = llm.invoke(state["question"]).content
    else:
        answer = improve(state["answer"])

    satisfied = len(answer) > 30
    return {"answer": answer, "satisfied": satisfied}


In [4]:
from langgraph.graph import END

def route(state: State):
    return END if state["satisfied"] else "agent"


In [5]:
from langgraph.graph import StateGraph

builder = StateGraph(State)

builder.add_node("agent", agent_node)
builder.set_entry_point("agent")

builder.add_conditional_edges("agent", route, {
    "agent": "agent",
    END: END
})

graph = builder.compile()


In [6]:
result = graph.invoke({"question": "Explain neural networks", "answer": "", "satisfied": False})
print(result["answer"])


Neural networks are a type of machine learning model inspired by the structure and function of the human brain. They consist of interconnected nodes, or neurons, that process and transmit information in a similar way to how biological neural networks function.

In a neural network, data is inputted into the first layer of neurons, called the input layer. These neurons then pass the data to the next layer of neurons, called the hidden layers, where the data is processed through a series of weighted connections and non-linear activation functions. Finally, the processed data is outputted by the neurons in the output layer.

During the training phase, a neural network learns to adjust the weights of the connections between neurons in order to minimize the difference between the predicted output and the actual output. This is done through a process called backpropagation, where the network uses an optimization algorithm, such as gradient descent, to update the weights based on the error be